# 0. 환경설정

In [ ]:
import re
import os
import logging
import json
from pymongo import MongoClient
from dotenv import load_dotenv
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 환경 변수 로드 및 로깅 설정
load_dotenv()
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 로그 저장 디렉토리 설정
LOG_DIR = "chat_logs"
os.makedirs(LOG_DIR, exist_ok=True)

# MongoDB 클라이언트 설정
MONGO_IP = os.getenv("MONGO_IP")
MONGO_PORT = int(os.getenv("MONGO_PORT"))
MONGO_USER = os.getenv("MONGO_USER")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")

# 연결 URI 생성
mongo_uri = f"mongodb://{MONGO_USER}:{MONGO_PASSWORD}@{MONGO_IP}:{MONGO_PORT}/?authSource=admin"
client = MongoClient(mongo_uri)

db = client['chatbot_db']
collection = db['processed_chats']

In [9]:
#----1. 모델 정의----
model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0
)

# 1. 전처리

### 1-1. txt -> xlsx

In [ ]:
# txt -> df

import pandas as pd
import os

# 원본 데이터가 있는 상위 폴더 경로
root_folder_path = "/Users/sdyplum/Desktop/DSCAP/Whatsapp_Crawling_Data/All_chats/Chats"

# 엑셀 파일을 저장할 폴더 경로
output_folder_path = "/Users/sdyplum/Desktop/DSCAP/DataScience_Capstone/data_preprocessing"

# 1. os.walk()로 상위 폴더부터 모든 하위 폴더 순회
# root: 현재 순회 중인 폴더 경로
# dirs: 현재 폴더에 있는 하위 폴더들 리스트
# files: 현재 폴더에 있는 파일들 리스트

for root, dirs, files in os.walk(root_folder_path):
    data = [] # 현재 폴더의 데이터를 담을 리스트 (매번 초기화)
    
    # 3. 현재 폴더 파일 순회
    for file_name in files:
        if file_name.endswith(".txt"):
            file_path = os.path.join(root, file_name)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                data.append([file_name, content])
            except Exception as e:
                print(f"파일 읽기 오류 '{file_path}': {e}")
    
    # 4. 현재 폴더에 처리할 txt 파일이 있었다면 데이터프레임 생성 및 저장
    if data:
        current_folder_name = os.path.basename(root)
        df = pd.DataFrame(data, columns=["file_name", "content"])
        output_file_path = os.path.join(output_folder_path, f"{current_folder_name}.xlsx")
    
        df.to_excel(output_file_path, index=False, engine="openpyxl")
        print(f"'{current_folder_name}' 폴더의 txt 파일들을 '{current_folder_name}.xlsx'로 저장 완료")

'whatsapp_exports_Navid' 폴더의 txt 파일들을 'whatsapp_exports_Navid.xlsx'로 저장 완료
'whatsapp_exports_Hadi' 폴더의 txt 파일들을 'whatsapp_exports_Hadi.xlsx'로 저장 완료
'whatsapp_exports_Hassan' 폴더의 txt 파일들을 'whatsapp_exports_Hassan.xlsx'로 저장 완료
'whatsapp_exports_Jalral' 폴더의 txt 파일들을 'whatsapp_exports_Jalral.xlsx'로 저장 완료
'whatsapp_exports_Haroon' 폴더의 txt 파일들을 'whatsapp_exports_Haroon.xlsx'로 저장 완료
'whatsapp_exports_Anjila' 폴더의 txt 파일들을 'whatsapp_exports_Anjila.xlsx'로 저장 완료
'whatsapp_exports_Omid' 폴더의 txt 파일들을 'whatsapp_exports_Omid.xlsx'로 저장 완료
'whatsapp_exports_Bhram' 폴더의 txt 파일들을 'whatsapp_exports_Bhram.xlsx'로 저장 완료


In [ ]:
cwd = os.getcwd()

folder_path = cwd
output_file = os.path.join(cwd, 'merged_files.xlsx')

# 1. .xlsx 확장자를 가진 모든 파일 목록 가져오기
all_files = os.listdir(folder_path)
excel_files = [f for f in all_files if f.endswith('.xlsx')]

if not excel_files:
    print("지정된 폴더에 엑셀 파일(.xlsx)이 없습니다.")
else:
    print(f"총 {len(excel_files)}개의 엑셀 파일을 병합합니다.")
    
    # 2. 각 엑셀 파일을 순서대로 읽어 데이터프레임 리스트에 추가
    df_list = []
    for file_name in excel_files:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_excel(file_path)
        df_list.append(df)
        print(f" - '{file_name}' 파일 로드 완료")

    # 3. 데이터프레임 리스트를 하나로 합치기
    merged_df = pd.concat(df_list, ignore_index=True) 

    # 4. 병합된 데이터프레임을 새로운 엑셀 파일로 저장
    merged_df.to_excel(output_file, index=False, engine='openpyxl')

    print(f"결과가 '{output_file}' 파일에 저장되었습니다.")

총 8개의 엑셀 파일을 병합합니다.
 - 'whatsapp_exports_Anjila.xlsx' 파일 로드 완료
 - 'whatsapp_exports_Navid.xlsx' 파일 로드 완료
 - 'whatsapp_exports_Haroon.xlsx' 파일 로드 완료
 - 'whatsapp_exports_Bhram.xlsx' 파일 로드 완료
 - 'whatsapp_exports_Jalral.xlsx' 파일 로드 완료
 - 'whatsapp_exports_Hadi.xlsx' 파일 로드 완료
 - 'whatsapp_exports_Omid.xlsx' 파일 로드 완료
 - 'whatsapp_exports_Hassan.xlsx' 파일 로드 완료
결과가 '/Users/sdyplum/Desktop/DSCAP/DataScience_Capstone/data_preprocessing/merged_files.xlsx' 파일에 저장되었습니다.


### 1-2. 시스템 메시지 + 이모티콘 제거

In [ ]:
# 데이터 불러오기
df = pd.read_excel("merged_files.xlsx")

def clean_chat_text(text: str) -> str:
    # 1. 불필요한 안내 문구 제거
    text = re.sub(r"Messages and calls are end-to-end encrypted.*?\n", "", text)
    text = re.sub(r"Welcome to the chat:.*?\n", "", text)

    # 2. 특수문자/이모지 제거
    text = re.sub(r"[^\w\s\[\]:\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]", " ", text)
    # (아랍/파슈토/다리어 문자 범위는 보존)
    
    # 3. 줄바꿈 제거 
    text = text.replace("\n", "")

    return text

# content 열 전처리 적용
df["clean_content"] = df["content"].apply(clean_chat_text)

# 중복 제거
df = df.drop_duplicates(subset=["clean_content"]).reset_index(drop=True)

### 1-3. 기계적인 메시지 삭제

In [ ]:
# 삭제할 키워드 목록
unwanted_keywords = [
    "wds-ill-ads-WA.st0",
    "chat-filled-refreshed2",
    "megaphone-refreshed-32"
]

# 삭제할 키워드들을 '|'로 묶어 하나의 검색 정규식으로 만들기
search_pattern = '|'.join(unwanted_keywords)

# 1. 키워드가 포함된 행을 찾고, 그 결과를 '~'로 뒤집고, 키워드가 포함되지 않은 행만 선택하여 바로 새로운 데이터프레임 생성
df_filtered = df[~df.apply(lambda x: x.astype(str).str.contains(search_pattern, na=False)).any(axis=1)]

# 2. 전처리 이전 내용 저장된 컬럼 content 삭제
df_filtered = df_filtered.drop('content', axis=1)

# 3. 최종적으로 제거된 데이터 저장
df_filtered.to_excel("merged_cleaned.xlsx", index=False, engine="openpyxl")

print(df_filtered)

                file_name                                      clean_content
0     _93 77 594 1073.txt    93 77 594 1073 [4:56 PM]    93 77 594 1073: ...
1     _93 74 967 6191.txt    93 74 967 6191 [5:59 PM]    93 74 967 6191: ...
2     _93 79 795 2761.txt    93 79 795 2761 [11:13 PM]    93 79 795 2761:...
3     _93 78 811 6598.txt    93 78 811 6598 [2:26 PM]    93 78 811 6598: ...
4     _93 78 653 4544.txt    93 78 653 4544 [1:37 PM]    93 78 653 4544: ...
...                   ...                                                ...
3478  _93 77 109 4210.txt    93 77 109 4210 [1:38 PM]   You: برای شناخت ا...
3479  _93 79 296 9055.txt          93 79 296 9055 [12:30 PM]   You: 12:30 PM
3480  _93 78 926 2123.txt    93 78 926 2123 [4:48 PM]   You: درود بر شما ...
3481  _93 74 982 7240.txt    93 74 982 7240 [8:41 PM]   You: نه محترمکار ...
3483  _93 70 663 4501.txt    93 70 663 4501 [11:41 PM]   You: سلام به تلگ...

[3018 rows x 2 columns]


### 1-4. AI 활용하여 사적인 대화 판별 후 삭제

In [19]:
df = pd.read_excel("merged_cleaned.xlsx")
data = df

system = """
당신은 데이터 감별사입니다. 입력받은 파일의 데이터 중 사적인 데이터를 감별하여 반환합니다. 
사적인 대화는 모르는 사람과 대화하는 것이 아닌, 친근한 대화를 의미합니다.
사적인 대화라고 판단된다면 해당하는 행의 번호를 반환합니다. 
"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{data}")])
chain = prompt | model | StrOutputParser()

out = chain.invoke({"data": data})
out

2025-09-16 21:55:41,456 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'사적인 대화라고 판단되는 행의 번호는 다음과 같습니다:\n\n- 3015\n- 3016\n- 3017'

# df -> JSON

### 2-1. JSON 형태로 변환 테스트

In [ ]:
# 1. 엑셀 파일 읽기
df1 = pd.read_excel("merged_cleaned.xlsx")
data = df1[:3]

# 2. 프롬프트 템플릿 수정
system_template = """
당신은 텍스트 형식의 대화 로그를 구조화된 JSON 데이터로 변환하는 역할입니다. 
아래에 제시된 규칙과 형식에 따라 주어진 대화 내용을 JSON으로 변환하세요.

### 최종 JSON 구조:
{{
    "name": "{file_name}",
    "content": [
        {{
            "name": "발신자 이름 또는 번호",
            "time": "발신 시간",
            "message": "메시지 내용"
        }}
    ]
}}

### 변환 규칙:
1. 최상위 'name': 전체 대화를 대표하는 이름으로, 제공된 {file_name} 값을 사용한다.
2. 'content' 배열: 아래 대화 내용의 각 줄을 하나의 JSON 객체로 변환하여 이 배열에 순서대로 추가한다.
3. 개별 메시지 객체 분석:
    * 'time': 각 줄의 시작 부분에 있는 대괄호 `[...]` 안의 시간 정보(예: '4 56 PM')를 추출하여 할당한다.
    * 'name': 시간 정보 바로 뒤에 나오는 발신자 정보('You' 또는 전화번호)를 추출하여 할당한다.
    * 'message': 발신자 정보 뒤에 나오는 해당 줄의 모든 나머지 텍스트를 메시지 내용으로 간주하여 할당한다.

### 변환할 대화 내용:
{conversation_text}
"""

prompt = ChatPromptTemplate.from_template(system_template)
chain = prompt | model | StrOutputParser()

# 3. 데이터프레임을 한 줄씩 처리
results = []
for index, row in data.iterrows():
    file_name = row['file_name']
    conversation = row['clean_content']

    out = chain.invoke({
        "file_name": re.sub(r'[^\d\s]', '', file_name),
        "conversation_text": conversation
    })
    results.append(out)
    print(f"--- 처리 완료: {file_name} ---")
    print(out)

2025-09-16 22:15:03,889 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


--- 처리 완료: _93 77 594 1073.txt ---
```json
{
    "name": "93 77 594 1073",
    "content": [
        {
            "name": "93 77 594 1073",
            "time": "4:56 PM",
            "message": "Hello  Can I get more info on this"
        },
        {
            "name": "You",
            "time": "4:56 PM",
            "message": "از تماس شما با کلیسای زنان افغانستان سپاس گزاریم  لطفا بفرمایید چگونه می توانیم کمکتان کنیم"
        },
        {
            "name": "93 77 594 1073",
            "time": "4:56 PM",
            "message": "خو"
        },
        {
            "name": "You",
            "time": "6:37 PM",
            "message": "خواهر  برادر عزیز این انجیل را مطالعه کنید و در تعلیمات و جلسات ما اشتراک کنید؟؟"
        },
        {
            "name": "You",
            "time": "6:37 PM",
            "message": "یوحنا دری pdf"
        },
        {
            "name": "You",
            "time": "6:37 PM",
            "message": "خواهر   برادر عزیز بعداز مطالعه این کتاب لطفاً از

2025-09-16 22:15:12,637 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


--- 처리 완료: _93 74 967 6191.txt ---
```json
{
    "name": "93 74 967 6191",
    "content": [
        {
            "name": "93 74 967 6191",
            "time": "5:59 PM",
            "message": "سلام"
        },
        {
            "name": "You",
            "time": "5:59 PM",
            "message": "از تماس شما با کلیسای زنان افغانستان سپاس گزاریم  لطفا بفرمایید چگونه می توانیم کمکتان کنیم"
        },
        {
            "name": "You",
            "time": "7:55 PM",
            "message": "خواهر  برادر عزیز این انجیل را مطالعه کنید و در تعلیمات و جلسات ما اشتراک کنید؟؟"
        },
        {
            "name": "You",
            "time": "7:55 PM",
            "message": "یوحنا دری pdf"
        },
        {
            "name": "You",
            "time": "7:55 PM",
            "message": "خواهر   برادر عزیز بعداز مطالعه این کتاب لطفاً از تلگرام پیام بگذارید؟ چو برنامه های تعلیمی و پرستشی و همه روی تلگرام است برکت خداوند بر شما باد شماره وتساپ  ادمین  0012044102606شماره تلگرام ادمین:

2025-09-16 22:15:23,469 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


--- 처리 완료: _93 79 795 2761.txt ---
```json
{
    "name": "93 79 795 2761",
    "content": [
        {
            "name": "93 79 795 2761",
            "time": "11:13 PM",
            "message": "Hello  Can I get more info on this"
        },
        {
            "name": "You",
            "time": "11:13 PM",
            "message": "از تماس شما با کلیسای زنان افغانستان سپاس گزاریم  لطفا بفرمایید چگونه می توانیم کمکتان کنیم"
        },
        {
            "name": "You",
            "time": "3:15 AM",
            "message": "خواهر  برادر عزیز این انجیل را مطالعه کنید و در تعلیمات و جلسات ما اشتراک کنید؟؟"
        },
        {
            "name": "You",
            "time": "3:15 AM",
            "message": "یوحنا دری pdf"
        },
        {
            "name": "You",
            "time": "3:15 AM",
            "message": "خواهر   برادر عزیز بعداز مطالعه این کتاب لطفاً از تلگرام پیام بگذارید؟ چو برنامه های تعلیمی و پرستشی و همه روی تلگرام است برکت خداوند بر شما باد شماره وتساپ  ادمین  

### 2-2. JSON 형태로 변환 후 MongoDB에 저장
- MongoDB에 미리 접속해야 함

In [ ]:
# --- 1. MongoDB 연결 설정 ---
db = client['chatbot_db']
collection = db['processed_chats']

# --- 2. 엑셀 파일 및 LLM 체인 준비 ---
df = pd.read_excel("merged_cleaned.xlsx")
data = df

system_template = """
당신은 텍스트 형식의 대화 로그를 구조화된 JSON 데이터로 변환하는 역할입니다. 
아래에 제시된 규칙과 형식에 따라 주어진 대화 내용을 JSON으로 변환하세요.

### 최종 JSON 구조:
{{
    "name": "{file_name}",
    "content": [
        {{
            "name": "발신자 이름 또는 번호",
            "time": "발신 시간",
            "message": "메시지 내용"
        }}
    ]
}}

### 변환 규칙:
1. 최상위 'name': 전체 대화를 대표하는 이름으로, 제공된 {file_name} 값을 사용한다.
2. 'content' 배열: 아래 대화 내용의 각 줄을 하나의 JSON 객체로 변환하여 이 배열에 순서대로 추가한다.
3. 개별 메시지 객체 분석:
    * 'time': 각 줄의 시작 부분에 있는 대괄호 `[...]` 안의 시간 정보(예: '4 56 PM')를 추출하여 할당한다.
    * 'name': 시간 정보 바로 뒤에 나오는 발신자 정보('You' 또는 전화번호)를 추출하여 할당한다.
    * 'message': 발신자 정보 뒤에 나오는 해당 줄의 모든 나머지 텍스트를 메시지 내용으로 간주하여 할당한다.

### 변환할 대화 내용:
{conversation_text}
"""

prompt = ChatPromptTemplate.from_template(system_template)
chain = prompt | model | StrOutputParser()

# --- 3. 데이터프레임을 한 줄씩 처리하고 결과를 리스트에 모으기 ---
documents_to_insert = []  # MongoDB에 저장할 문서(딕셔너리)들을 담을 리스트

for index, row in data.iterrows():
    file_name = row['file_name']
    conversation = row['clean_content']


    out = chain.invoke({
        "file_name": re.sub(r'[^\d\s]', '', file_name),
        "conversation_text": conversation
    })

    print(f"--- 처리 완료: {file_name} ---")

    # LLM 출력에서 순수 JSON을 추출하고 딕셔너리로 변환
    try:
        json_match = re.search(r'```json\s*(\{.*?\})\s*```', out, re.DOTALL)
        if json_match:
            json_string = json_match.group(1)
            py_dict = json.loads(json_string)
            documents_to_insert.append(py_dict)
        else:
             # ```json ``` 형식이 아닐 경우를 대비
            py_dict = json.loads(out)
            documents_to_insert.append(py_dict)
            
    except Exception as e:
        print(f"[에러] {file_name} 처리 중 JSON 변환 실패: {e}")


# --- 4. 루프 종료 후, 모아둔 모든 문서를 MongoDB에 한번에 저장 ---
if documents_to_insert: 
    try:
        result = collection.insert_many(documents_to_insert)
        print("\n" + "="*50)
        print(f"총 {len(result.inserted_ids)}개의 문서를 MongoDB에 성공적으로 저장했습니다.")
        print("="*50)
    except Exception as e:
        print(f"\n[에러] MongoDB에 데이터를 저장하는 중 실패했습니다: {e}")
else:
    print("\nMongoDB에 저장할 데이터가 없습니다.")

# --- 5. 연결 종료 ---
client.close()

In [ ]:
# --- 4. 루프 종료 후, 모아둔 모든 문서를 MongoDB에 한번에 저장 ---
if documents_to_insert: 
    try:
        result = collection.insert_many(documents_to_insert)
        print("\n" + "="*50)
        print(f"총 {len(result.inserted_ids)}개의 문서를 MongoDB에 성공적으로 저장했습니다.")
        print("="*50)
    except Exception as e:
        print(f"\n[에러] MongoDB에 데이터를 저장하는 중 실패했습니다: {e}")
else:
    print("\nMongoDB에 저장할 데이터가 없습니다.")

# --- 5. 연결 종료 ---
client.close()


총 3017개의 문서를 MongoDB에 성공적으로 저장했습니다.
